In [4]:
import os
import torch
import numpy as np
import torch.nn as nn
import muse.supplier as spr
import muse.processor2 as pcr
import muse.model2 as mdl
import muse.trainer as trn
import muse.visualizer as vis

import warnings
warnings.filterwarnings("ignore")

In [7]:
instrument = 'Piano'
filepath = "./../input/haydn/"
song_len = 200*1
stride = 200
batch_size = 1
latent_size = 20
device = trn.get_device()

In [6]:
### ===== Data Preprocessing ===== ###

seed_load = np.random.randint(0,999999999)
seed_load = 592643464 # set to seed that works

all_midis, filenames = pcr.get_midis(filepath) # load all .midi files
Corpus, instru2corpus = pcr.get_notes_batch(all_midis) # extract all notes and sort by instrument
Corpus, fmap, rmap = pcr.get_map(Corpus) # get forward-map and reverse-map from corpus
instru2corpus = pcr.remove_short(instru2corpus) # remove songs that are too short

X_train_melody, X_val_melody, X_train_offset, X_val_offset = pcr.train_test_split(instru2corpus, instrument, fmap, song_len, stride,\
                                                                                  seed=seed_load, process='center')
X_train_melody, X_val_melody = pcr.batchify(X_train_melody), pcr.batchify(X_val_melody) # reshape and turn into tensor
X_train_offset, X_val_offset = pcr.batchify(X_train_offset), pcr.batchify(X_val_offset) # reshape and turn into tensor

classes = len(set(Corpus)) # get number of unique keys

In [12]:
model = mdl.cnn_varautoencoder(1, song_len//200, classes)
G = trn.load_model('vae4_melody_CE', model, device)[0].decoder

In [38]:
def gen_generation_gan(model, rmap, classes, base=1, fname='new'):
    sample_latent = lambda batch_size, latent_size: torch.FloatTensor(batch_size, latent_size).uniform_(0, classes)
    latent = sample_latent(1, latent_size).type(torch.FloatTensor)
    new_train_melody1 = model(latent)
    scale = torch.max(new_train_melody1, dim=-1)[0].item()
    new_train_melody1 = new_train_melody1.multiply(classes)/scale
    new_train_melody2 = pcr.rmap_safe(rmap, new_train_melody1.view(-1).detach().numpy())
    const_offset = np.arange(len(new_train_melody2))
    
    pcr.gen_stream(new_train_melody2, const_offset, base=base, output=True,
               fname=fname)
    return new_train_melody1

In [40]:
for i in range(20):
    _ = gen_generation_gan(G, rmap, classes, base=1, fname='new_'+str(i))